# Quantum Circuits

- Calculation of energy on simulator / QPU using previously found QCC parameters>
- \> 8 qubit machine is not available as of now because of dedicated use.

In [7]:
import tequila as tq
from utility import *



from qiskit import IBMQ
IBMQ.save_account('')

## H$_4$

In [10]:
xyz_data = get_molecular_data('h4', geometry=90, xyz_format=True)

basis = '6-31g'
active = { 'AG':[0,1], 'B1U': [0], 'B2U': [0]}
h4 = tq.quantumchemistry.Molecule(geometry=xyz_data, basis_set = basis, active_orbitals=active)
H = h4.make_hamiltonian()
U = construct_QMF_ansatz(H.n_qubits)
print('Qubits required', H.n_qubits)


entangulars = ['X(0)Y(1)X(4)X(5)', 'X(2)Y(3)X(4)X(5)', 'X(0)Y(1)X(6)X(7)', 'X(2)Y(3)X(6)X(7)']
for i, entangular in enumerate(entangulars):
    a = tq.Variable(f"tau_{i}")
    U += tq.gates.ExpPauli(paulistring=tq.PauliString.from_string(entangular), angle=a)

E = tq.ExpectationValue(H=H, U=U)
vars = { 'beta_0': 3.141592653589793,  'gamma_0': 0.0,  'beta_1': 3.141592653589793,  'gamma_1': 0.0,  'beta_2': 3.141592653589793,  'gamma_2': 0.0,  'beta_3': 3.141592653589793,  'gamma_3': 0.0,  'beta_4': 0.0,  'gamma_4': 0.0,  'beta_5': 0.0,  'gamma_5': 0.0,  'beta_6': 0.0,  'gamma_6': 0.0,  'beta_7': 0.0,  'gamma_7': 0.0,  'tau_0': 0.001113151574588045,  'tau_1': 1.4945134312876838,  'tau_2': 0.03185012201085507,  'tau_3': 0.0018841847678164976}
print('Exact simulation: E=', tq.simulate(E, variables=vars))
print('QASM simulation (100 shots): E=', tq.simulate(E, variables=vars, samples=100, backend="qiskit", device='qasm_simulator'))
# print('QPU (100 shots): E=', tq.simulate(E, variables=vars, samples=100, backend="qiskit", device='ibmq_16_melbourne'))
circ = tq.circuit.compiler.compile_exponential_pauli_gate(U)
tq.draw(circ, backend="qiskit")


There are known issues with some psi4 methods and frozen virtual orbitals. Proceed with fingers crossed for hf.
Qubits required 8
Exact simulation: E= -1.9783834725768927
QASM simulation (100 shots): E= -2.0146303340333214
      ┌────────────────────┐┌─────────────────────┐    ┌───┐                   »
q_0: ─┤ RX(f((beta_0,))_0) ├┤ RZ(f((gamma_0,))_1) ├────┤ H ├──────■────────────»
      ├────────────────────┤├─────────────────────┤ ┌──┴───┴───┐┌─┴─┐          »
q_1: ─┤ RX(f((beta_1,))_2) ├┤ RZ(f((gamma_1,))_3) ├─┤ RX(pi/2) ├┤ X ├──■───────»
      ├────────────────────┤├─────────────────────┤ └──┬───┬───┘└───┘  │       »
q_2: ─┤ RX(f((beta_2,))_4) ├┤ RZ(f((gamma_2,))_5) ├────┤ H ├──────■────┼───────»
      ├────────────────────┤├─────────────────────┤ ┌──┴───┴───┐┌─┴─┐  │       »
q_3: ─┤ RX(f((beta_3,))_6) ├┤ RZ(f((gamma_3,))_7) ├─┤ RX(pi/2) ├┤ X ├──┼───────»
      ├────────────────────┤├─────────────────────┤ └──┬───┬───┘└───┘┌─┴─┐     »
q_4: ─┤ RX(f((beta_4,))_8) ├┤ RZ(f((gamma_4,))_9

## LiH

In [11]:
xyz_data = get_molecular_data('lih', geometry=1.5, xyz_format=True)
basis='6-31g'
active = {'A1': [0, 1], 'B1': [0], 'B2': [0]}
lih = tq.quantumchemistry.Molecule(geometry=xyz_data, basis_set=basis, active_orbitals=active)

hf_reference = hf_occ(2*lih.n_orbitals, lih.n_electrons)

H = lih.make_hamiltonian()
U = construct_QMF_ansatz(H.n_qubits)
print('Qubits required', H.n_qubits)


entangulars = ['X(2)Y(3)X(4)X(5)', 'X(2)Y(3)X(6)X(7)']
for i, entangular in enumerate(entangulars):
    a = tq.Variable(f"tau_{i}")
    U += tq.gates.ExpPauli(paulistring=tq.PauliString.from_string(entangular), angle=a)

E = tq.ExpectationValue(H=H, U=U)
vars = { 'beta_0': 3.141592653589793,  'gamma_0': 0.0,  'beta_1': 3.141592653589793,  'gamma_1': 0.0,  'beta_2': 3.1415924151712136,  'gamma_2': 0.0,  'beta_3': 3.1415924151712136,  'gamma_3': 0.0,  'beta_4': 0.0,  'gamma_4': 0.0,  'beta_5': 0.0,  'gamma_5': 0.0,  'beta_6': 0.0,  'gamma_6': 0.0,  'beta_7': 0.0,  'gamma_7': 0.0,  'tau_0': 0.02792620808183044,  'tau_1': 0.02792620808183044}
print('Exact simulation: E=', tq.simulate(E, variables=vars))
print('QASM simulation (100 shots): E=', tq.simulate(E, variables=vars, samples=100, backend="qiskit", device='qasm_simulator'))
# print('QPU (100 shots): E=', tq.simulate(E, variables=vars, samples=100, backend="qiskit", device='ibmq_16_melbourne'))
circ = tq.circuit.compiler.compile_exponential_pauli_gate(U)
tq.draw(circ, backend="qiskit")


There are known issues with some psi4 methods and frozen virtual orbitals. Proceed with fingers crossed for hf.
Qubits required 8
Exact simulation: E= -7.977129172029752
QASM simulation (100 shots): E= -7.979885671662881
      ┌────────────────────┐┌─────────────────────┐                            »
q_0: ─┤ RX(f((beta_0,))_0) ├┤ RZ(f((gamma_0,))_1) ├────────────────────────────»
      ├────────────────────┤├─────────────────────┤                            »
q_1: ─┤ RX(f((beta_1,))_2) ├┤ RZ(f((gamma_1,))_3) ├────────────────────────────»
      ├────────────────────┤├─────────────────────┤    ┌───┐                   »
q_2: ─┤ RX(f((beta_2,))_4) ├┤ RZ(f((gamma_2,))_5) ├────┤ H ├──────■────────────»
      ├────────────────────┤├─────────────────────┤ ┌──┴───┴───┐┌─┴─┐          »
q_3: ─┤ RX(f((beta_3,))_6) ├┤ RZ(f((gamma_3,))_7) ├─┤ RX(pi/2) ├┤ X ├──■───────»
      ├────────────────────┤├─────────────────────┤ └──┬───┬───┘└───┘┌─┴─┐     »
q_4: ─┤ RX(f((beta_4,))_8) ├┤ RZ(f((gamma_4,))_9) 

## NH$_3$

In [12]:
xyz_data = get_molecular_data('nh3', geometry=1.5, xyz_format=True)
basis='6-31g'
active = {'A\'': [2, 3], 'A"': [0, 1]}
nh3 = tq.quantumchemistry.Molecule(geometry=xyz_data, basis_set=basis, active_orbitals=active)
H = nh3.make_hamiltonian()
U = construct_QMF_ansatz(H.n_qubits)
print('Qubits required', H.n_qubits)


entangulars = ['X(0)Y(1)X(6)X(7)', 'X(4)Y(5)X(6)X(7)', 'X(2)Y(3)X(6)X(7)', 'X(2)Y(5)X(6)X(7)']
for i, entangular in enumerate(entangulars):
    a = tq.Variable(f"tau_{i}")
    U += tq.gates.ExpPauli(paulistring=tq.PauliString.from_string(entangular), angle=a)

E = tq.ExpectationValue(H=H, U=U)
vars = { 'beta_0': 3.141592653589793,  'gamma_0': 0.0,  'beta_1': 3.141592653589793,  'gamma_1': 0.0,  'beta_2': 3.141592653589793,  'gamma_2': 0.0,  'beta_3': 3.141592653589793,  'gamma_3': 0.0,  'beta_4': 3.141592653589793,  'gamma_4': 0.0,  'beta_5': 3.141592653589793,  'gamma_5': 0.0,  'beta_6': 0.0,  'gamma_6': 0.0,  'beta_7': 0.0,  'gamma_7': 0.0,  'tau_0': 0.28104602086856795,  'tau_1': 0.0792502789439926,  'tau_2': 0.0433205302668355,  'tau_3': 0.040185506394641614,  'tau_4': -0.040185506394641614}
print('Exact simulation: E=', tq.simulate(E, variables=vars))
print('QASM simulation (100 shots): E=', tq.simulate(E, variables=vars, samples=100, backend="qiskit", device='qasm_simulator'))
# print('QPU (100 shots): E=', tq.simulate(E, variables=vars, samples=100, backend="qiskit", device='ibmq_16_melbourne'))
circ = tq.circuit.compiler.compile_exponential_pauli_gate(U)
tq.draw(circ, backend="qiskit")

There are known issues with some psi4 methods and frozen virtual orbitals. Proceed with fingers crossed for hf.
Qubits required 8
Exact simulation: E= -55.91525770733235
QASM simulation (100 shots): E= -55.900631573708516
      ┌────────────────────┐┌─────────────────────┐    ┌───┐                   »
q_0: ─┤ RX(f((beta_0,))_0) ├┤ RZ(f((gamma_0,))_1) ├────┤ H ├──────■────────────»
      ├────────────────────┤├─────────────────────┤ ┌──┴───┴───┐┌─┴─┐          »
q_1: ─┤ RX(f((beta_1,))_2) ├┤ RZ(f((gamma_1,))_3) ├─┤ RX(pi/2) ├┤ X ├──■───────»
      ├────────────────────┤├─────────────────────┤ └──┬───┬───┘└───┘  │       »
q_2: ─┤ RX(f((beta_2,))_4) ├┤ RZ(f((gamma_2,))_5) ├────┤ H ├──────■────┼───────»
      ├────────────────────┤├─────────────────────┤ ┌──┴───┴───┐┌─┴─┐  │       »
q_3: ─┤ RX(f((beta_3,))_6) ├┤ RZ(f((gamma_3,))_7) ├─┤ RX(pi/2) ├┤ X ├──┼───────»
      ├────────────────────┤├─────────────────────┤ └──┬───┬───┘└───┘  │       »
q_4: ─┤ RX(f((beta_4,))_8) ├┤ RZ(f((gamma_4,))_9)

## N$_2$

In [13]:
xyz_data = get_molecular_data('n2', geometry=1.5, xyz_format=True)
basis='6-31g'
active = {'AG': [1, 2], 'B1U': [1], 'B2U': [1]}
n2 = tq.quantumchemistry.Molecule(geometry=xyz_data, basis_set=basis, active_orbitals=active)
H = n2.make_hamiltonian()
U = construct_QMF_ansatz(H.n_qubits)
print('Qubits required', H.n_qubits)


entangulars =[ 'X(2)Y(3)X(6)X(7)', 'X(0)Y(1)X(6)X(7)', 'X(4)Y(5)X(6)X(7)', 'X(0)Y(5)X(6)X(7)', 'X(1)Y(4)X(6)X(7)']
for i, entangular in enumerate(entangulars):
    a = tq.Variable(f"tau_{i}")
    U += tq.gates.ExpPauli(paulistring=tq.PauliString.from_string(entangular), angle=a)

E = tq.ExpectationValue(H=H, U=U)
vars = { 'beta_0': 3.141592653589793,  'gamma_0': 0.0,  'beta_1': 3.141592653589793,  'gamma_1': 0.0,  'beta_2': 3.141592653589793,  'gamma_2': 0.0,  'beta_3': 3.141592653589793,  'gamma_3': 0.0,  'beta_4': 3.141592653589793,  'gamma_4': 0.0,  'beta_5': 3.141592653589793,  'gamma_5': 0.0,  'beta_6': 0.0,  'gamma_6': 0.0,  'beta_7': 0.0,  'gamma_7': 0.0,  'tau_0': 0.014515100229508337,  'tau_1': 0.010657633274739736,  'tau_2': 0.007544088839867546,  'tau_3': 0.002358922869994376,  'tau_4': -0.0023589228699943754}
print('Exact simulation: E=', tq.simulate(E, variables=vars))
print('QASM simulation (100 shots): E=', tq.simulate(E, variables=vars, samples=100, backend="qiskit", device='qasm_simulator'))
# print('QPU (100 shots): E=', tq.simulate(E, variables=vars, samples=100, backend="qiskit", device='ibmq_16_melbourne'))
circ = tq.circuit.compiler.compile_exponential_pauli_gate(U)
tq.draw(circ, backend="qiskit")


There are known issues with some psi4 methods and frozen virtual orbitals. Proceed with fingers crossed for hf.
Qubits required 8
Exact simulation: E= -108.62442715453423
QASM simulation (100 shots): E= -108.64863375678442
      ┌────────────────────┐┌─────────────────────┐    ┌───┐                   »
q_0: ─┤ RX(f((beta_0,))_0) ├┤ RZ(f((gamma_0,))_1) ├────┤ H ├──────■────────────»
      ├────────────────────┤├─────────────────────┤ ┌──┴───┴───┐┌─┴─┐          »
q_1: ─┤ RX(f((beta_1,))_2) ├┤ RZ(f((gamma_1,))_3) ├─┤ RX(pi/2) ├┤ X ├──────────»
      ├────────────────────┤├─────────────────────┤ └──┬───┬───┘└───┘          »
q_2: ─┤ RX(f((beta_2,))_4) ├┤ RZ(f((gamma_2,))_5) ├────┤ H ├──────■────────────»
      ├────────────────────┤├─────────────────────┤ ┌──┴───┴───┐┌─┴─┐          »
q_3: ─┤ RX(f((beta_3,))_6) ├┤ RZ(f((gamma_3,))_7) ├─┤ RX(pi/2) ├┤ X ├──■───────»
      ├────────────────────┤├─────────────────────┤ └──┬───┬───┘└───┘  │       »
q_4: ─┤ RX(f((beta_4,))_8) ├┤ RZ(f((gamma_4,))_9